# Notebook: Create OPA2VEC associations file
<b>Author</b>: Ian Coleman <br/>
<b>Function</b>: Get genes associated with diseases (disgenet) and chemicals (ctdbase), take their go functions and assign to the associated chem/dis in an associations file

In [1]:
import pandas as pd
import numpy as np
import subprocess

### 1. Get Uniprot ID set for each Gene 

In [2]:
# CHEMICALS
# Import CTD file to get the geneIDs we want 
df_cg = pd.read_csv('../ctd-to-nt/csvs/CTD_chem_gene_ixns.csv', skiprows=27)
df_cg = df_cg.drop(0)

# Cut down to just humans if you want to
df_cg = df_cg.loc[df_cg['OrganismID'] == 9606.0]

# Need to change float to int for the url to work
df_cg['GeneID'] = df_cg.GeneID.astype(int)

In [3]:
# DISEASES
# Import DisGeNet with disease IDs and Gene IDs
df_cgd = pd.read_csv('../disgenet-nt/input_tsvs/curated_gene_disease_associations.tsv', sep='\t')

# Limit by evidence Score 
df_cgd = df_cgd[df_cgd.score >= 0.42]

In [4]:
# # Create list of associated GeneIDs for each of Chemicals and Diseases
cgene_ids = df_cg.GeneID.unique()
dgene_ids = df_cgd.geneId.unique()

# Combine them
all_genes = set(cgene_ids).union(set(dgene_ids))

In [5]:
# Write combined gene ids to file
with open('geneIDs.txt', 'w') as f:
    for item in all_genes:
        f.write("%s\n" % item)

In [ ]:
# NOTE the next step is MANUAL
# You need to go to https://www.uniprot.org/uploadlists/ and give it the created geneIDs.txt file, ask it to convert
# entrez gene to uniprot ID. Then download this as uniprotIDs.txt (as uncompressed, mapping table)

##### Import the list of uniprot IDs

In [6]:
# Import manually generated file of geneID --> uniprotID
df_uni_ids = pd.read_csv('uniprotIDs.txt', sep='\t')
df_uni_ids.columns = ['GeneID', 'UniprotID']
df_uni_ids['GeneID'] = df_uni_ids.GeneID.astype(str)

In [7]:
# Some of the GeneIds are actually a couple of IDs on one row. Split them into multiple rows
df_uni_ids = pd.concat([pd.Series(row['UniprotID'], row['GeneID'].split(','))              
                    for _, row in df_uni_ids.iterrows()]).reset_index()
df_uni_ids.columns = ['GeneID', 'UniprotID']

##### Create merged df to enable grouping by chemicalID and diseaseID

In [8]:
# Create mapping file of gene ID to chem ID... and gene ID to disease ID 
cg_dict = dict(zip(df_cg.GeneID, df_cg.ChemicalID))
cgd_dict = dict(zip(df_cgd.geneId, df_cgd.diseaseId))

In [9]:
df_uni_ids.head()

,GeneID,UniprotID
0,1,P04217
1,1,V9HWD8
2,2,P01023
3,131076,C9JQ41
4,131076,Q4VC31


In [10]:
df_uni_ids_d = df_uni_ids.copy() # make a copy for diseases

In [11]:
# Apply the mapping, thereby creating chemical column
df_uni_ids['ChemicalID'] = df_uni_ids.GeneID
df_uni_ids['ChemicalID'] = df_uni_ids['ChemicalID'].astype(int).map(cg_dict)

In [12]:
# Apply the mapping, thereby creating disease column (replace is slower than map TODO use map method)
df_uni_ids_d['DiseaseID'] = df_uni_ids_d.GeneID
df_uni_ids_d['DiseaseID'] = df_uni_ids_d['DiseaseID'].astype(int).map(cgd_dict)

##### Group by Chem ID

In [13]:
df_uni_ids.dropna().shape # 33381

(33381, 3)

In [14]:
df_uni_ids_d.dropna().shape #2914

(2914, 3)

### 2. Mine goa file, attaching go function for each uniprot ID to the parent chemical/gene

In [86]:
# df_uni_ids_d.head()

In [46]:
# import goa file (uniprot ID to go_functions)
go_funcs = pd. read_csv('../goa_human.gaf', header=None, skiprows=30, sep='\t')

/home/ian/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [87]:
# go_funcs.head()

In [48]:
# Cut out all cols except uniprot ids and go_funcs, rename these
go_funcs = go_funcs.rename(columns={ go_funcs.columns[1]: "UniprotID" })
go_funcs = go_funcs.rename(columns={ go_funcs.columns[4]: "gofunc" })
col_list = ['UniprotID', 'gofunc']
df_go = go_funcs[col_list]

In [52]:
df_uni_ids_d = df_uni_ids_d.merge(df_go, on='UniprotID', how='outer').dropna()
df_uni_ids = df_uni_ids.merge(df_go, on='UniprotID', how='outer').dropna()

### 3. Prep and write to output file 

In [88]:
# df_uni_ids_d.head()

In [74]:
# Create a col with the full go url
df_uni_ids['go_url'] = '<' + 'http://purl.obolibrary.org/obo/' + df_uni_ids.gofunc.str.replace(':', '_')  + '>'
df_uni_ids_d['go_url'] =  '<' + 'http://purl.obolibrary.org/obo/' + df_uni_ids_d.gofunc.str.replace(':', '_')  + '>'

In [77]:
# Grab just the columns to output
col_list_c = ['ChemicalID', 'go_url']
col_list_d = ['DiseaseID', 'go_url']
df_c = df_uni_ids[col_list_c]
df_d = df_uni_ids_d[col_list_d]

In [89]:
# df_c.head()

In [84]:
# Output an association file for each of chem and dis
np.savetxt(r'associations_c.txt', df_c.values, fmt='%s')
np.savetxt(r'associations_d.txt', df_d.values, fmt='%s')

In [85]:
# Merge these two into one single file
subprocess.call('cat associations* > myassociations', shell=True)

0